In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [ ]:
! pip install -U 'scikit-learn<0.24'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.8 MB 29.5 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.4 requires scikit-learn>=1.0.0, but you have scikit-learn 0.23.2 which is incompatible.
imbalanced-learn 0.8.1 requires scikit-learn>=0.24, but you have scikit-learn 0.23.2 which is incompatible.


In [ ]:
f=open("train.txt", "r")
l=[]
lines=[]
for i in f.readlines():
  if i[0]=="\n":
    lines.append(l)
    l=[]
  else:
    l.append(i)
f.close()
lines[1]

['EU NNP B-NP B-ORG\n',
 'rejects VBZ B-VP O\n',
 'German JJ B-NP B-MISC\n',
 'call NN I-NP O\n',
 'to TO B-VP O\n',
 'boycott VB I-VP O\n',
 'British JJ B-NP B-MISC\n',
 'lamb NN I-NP O\n',
 '. . O O\n']

In [ ]:
f=open("test.txt", "r")
l=[]
test_lines=[]
for i in f.readlines():
  if i[0]=="\n":
    test_lines.append(l)
    l=[]
  else:
    l.append(i)
f.close()
test_lines[1]

['SOCCER NN B-NP O\n',
 '- : O O\n',
 'JAPAN NNP B-NP B-LOC\n',
 'GET VB B-VP O\n',
 'LUCKY NNP B-NP O\n',
 'WIN NNP I-NP O\n',
 ', , O O\n',
 'CHINA NNP B-NP B-PER\n',
 'IN IN B-PP O\n',
 'SURPRISE DT B-NP O\n',
 'DEFEAT NN I-NP O\n',
 '. . O O\n']

In [ ]:
def fun(con_list):
  line=[]
  for i in con_list:
    c=[]
    for j in i:
      j=j.strip("\n")
      if j!="":
        j=j.split(" ")
        if j[0]!="-DOCSTART-":
          c.append(tuple(j))
    if c!=[]:
      line.append(c)
  return line

In [ ]:
c=fun(lines)
t=fun(test_lines)

In [ ]:
c[0]

[('EU', 'NNP', 'B-NP', 'B-ORG'),
 ('rejects', 'VBZ', 'B-VP', 'O'),
 ('German', 'JJ', 'B-NP', 'B-MISC'),
 ('call', 'NN', 'I-NP', 'O'),
 ('to', 'TO', 'B-VP', 'O'),
 ('boycott', 'VB', 'I-VP', 'O'),
 ('British', 'JJ', 'B-NP', 'B-MISC'),
 ('lamb', 'NN', 'I-NP', 'O'),
 ('.', '.', 'O', 'O')]

In [ ]:
def text_features(sent, i):
  word, pos_tag, chunk_tag, label=sent[i]
  d=dict()
  d["is_digit"]=word.isdigit()
  d["is_upper"]=word.isupper()
  d["bias"]= 1.0
  d["lower"]=word.lower()
  d["is_title"]=word.istitle()
  d["pos_tag"]=pos_tag
  d['word[-3:]']= word[-3:]
  d['word[-2:]']= word[-2:]

  if i > 0:
      try:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        d.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
      except:
        print(i,sent)
  else:
      d['BOS'] = True

  if i < len(sent)-1:
      
      word1 = sent[i+1][0]
      postag1 = sent[i+1][1]
      d.update({
          '+1:word.lower()': word1.lower(),
          '+1:word.istitle()': word1.istitle(),
          '+1:word.isupper()': word1.isupper(),
          '+1:postag': postag1,
          '+1:postag[:2]': postag1[:2],
    })

  else:
      d['EOS'] = True


  return d

def extract_features(data):
  features=[]
  labels=[]
  for i in data:
    feature=[]
    label=[]
    for j in range(len(i)):
      feature.append(text_features(i,j))
      label.append(i[j][-1])
    features.append(feature)
    labels.append(label)
  return features, labels

In [ ]:
X_train, y_train=extract_features(c)
X_test, y_test=extract_features(t)

In [ ]:
X_train[1]

[{'+1:postag': 'NNP',
  '+1:postag[:2]': 'NN',
  '+1:word.istitle()': True,
  '+1:word.isupper()': False,
  '+1:word.lower()': 'blackburn',
  'BOS': True,
  'bias': 1.0,
  'is_digit': False,
  'is_title': True,
  'is_upper': False,
  'lower': 'peter',
  'pos_tag': 'NNP',
  'word[-2:]': 'er',
  'word[-3:]': 'ter'},
 {'-1:postag': 'NNP',
  '-1:postag[:2]': 'NN',
  '-1:word.istitle()': True,
  '-1:word.isupper()': False,
  '-1:word.lower()': 'peter',
  'EOS': True,
  'bias': 1.0,
  'is_digit': False,
  'is_title': True,
  'is_upper': False,
  'lower': 'blackburn',
  'pos_tag': 'NNP',
  'word[-2:]': 'rn',
  'word[-3:]': 'urn'}]

In [ ]:
final_data = []
for i in range(len(c)):
 final_data.append(c[i][0])
final_data

[('EU', 'NNP', 'B-NP', 'B-ORG'),
 ('Peter', 'NNP', 'B-NP', 'B-PER'),
 ('BRUSSELS', 'NNP', 'B-NP', 'B-LOC'),
 ('The', 'DT', 'B-NP', 'O'),
 ('Germany', 'NNP', 'B-NP', 'B-LOC'),
 ('"', '"', 'O', 'O'),
 ('He', 'PRP', 'B-NP', 'O'),
 ('He', 'PRP', 'B-NP', 'O'),
 ('Fischler', 'JJR', 'B-NP', 'B-PER'),
 ('But', 'CC', 'O', 'O'),
 ('Spanish', 'NNP', 'B-NP', 'B-MISC'),
 ('.', '.', 'O', 'O'),
 ('Only', 'RB', 'B-NP', 'O'),
 ('The', 'DT', 'B-NP', 'O'),
 ('Sheep', 'NNP', 'B-NP', 'O'),
 ('British', 'JJ', 'B-NP', 'B-MISC'),
 ('"', '"', 'O', 'O'),
 ('Bonn', 'NNP', 'B-NP', 'B-LOC'),
 ('Germany', 'NNP', 'B-NP', 'B-LOC'),
 ('It', 'PRP', 'B-NP', 'O'),
 ('Rare', 'NNP', 'B-NP', 'O'),
 ('LONDON', 'NNP', 'B-NP', 'B-LOC'),
 ('A', 'DT', 'B-NP', 'O'),
 ('A', 'DT', 'B-NP', 'O'),
 ('At', 'IN', 'B-PP', 'O'),
 ('Buyers', 'NNS', 'B-NP', 'O'),
 ('They', 'PRP', 'B-NP', 'O'),
 ('The', 'DT', 'B-NP', 'O'),
 ('China', 'NNP', 'B-NP', 'B-LOC'),
 ('BEIJING', 'VBG', 'B-VP', 'B-LOC'),
 ('China', 'NNP', 'B-NP', 'B-LOC'),
 ('Speakin

In [ ]:
!pip install sklearn_crfsuite
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 965 kB 24.8 MB/s 


In [ ]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=False,
)
crf.fit(X_train, y_train);

In [ ]:
labels = list(crf.classes_)
labels.remove('O')
labels

['B-ORG', 'B-MISC', 'B-PER', 'I-PER', 'B-LOC', 'I-ORG', 'I-MISC', 'I-LOC']

In [ ]:
y_pred=crf.predict(X_test)

In [ ]:
sorted_labels = sorted(
    labels, 
    key=lambda name: (name[1:], name[0])
)

In [ ]:
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:70: FutureWarning: Pass labels=['B-LOC', 'I-LOC', 'B-MISC', 'I-MISC', 'B-ORG', 'I-ORG', 'B-PER', 'I-PER'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


              precision    recall  f1-score   support

       B-LOC      0.856     0.810     0.832      1668
       I-LOC      0.764     0.619     0.684       257
      B-MISC      0.807     0.758     0.782       702
      I-MISC      0.664     0.657     0.660       216
       B-ORG      0.768     0.727     0.747      1661
       I-ORG      0.665     0.744     0.702       835
       B-PER      0.831     0.858     0.844      1617
       I-PER      0.876     0.949     0.911      1156

   micro avg      0.803     0.801     0.802      8112
   macro avg      0.779     0.765     0.770      8112
weighted avg      0.804     0.801     0.801      8112



In [ ]:
import scipy
params_space = {
 'c1': np.linspace(0, np.minimum(scipy.stats.expon(scale=0.5).dist.b, 5))[:5],
 'c2': np.linspace(0, np.minimum(scipy.stats.expon(scale=0.05).dist.b, 5))[:5],
}

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
f1_scorer = make_scorer(metrics.flat_f1_score, average='macro')
gs = GridSearchCV(crf, params_space,
 cv=3,
 verbose=1,
 n_jobs=-1,
 scoring=f1_scorer)


In [ ]:
gs.fit(X_train, y_train)

Fitting 3 folds for each of 25 candidates, totalling 75 fits


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:213: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed: 21.6min
[Parallel(n_jobs=-1)]: Done  75 out of  75 | elapsed: 34.7min finished


GridSearchCV(cv=3,
             estimator=CRF(algorithm='lbfgs', all_possible_transitions=False,
                           c1=0.1, c2=0.1, keep_tempfiles=None,
                           max_iterations=100),
             n_jobs=-1,
             param_grid={'c1': array([0.        , 0.10204082, 0.20408163, 0.30612245, 0.40816327]),
                         'c2': array([0.        , 0.10204082, 0.20408163, 0.30612245, 0.40816327])},
             scoring=make_scorer(flat_f1_score, average=macro), verbose=1)

In [ ]:
g_crf= gs.best_estimator_
print('best params:', gs.best_params_)
print('best CV score:', gs.best_score_)

best params: {'c1': 0.10204081632653061, 'c2': 0.20408163265306123}
best CV score: 0.8405999900151727


In [ ]:
from collections import Counter

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("\nTop likely transitions:")
print_transitions(Counter(g_crf.transition_features_).most_common()[:5])


Top likely transitions:
B-PER  -> I-PER   8.259603
B-LOC  -> I-LOC   7.470484
B-ORG  -> I-ORG   7.257616
B-MISC -> I-MISC  6.757667
I-ORG  -> I-ORG   6.558049


In [ ]:
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-8s %s" % (weight, label, attr))    

print("\nTop positive:")
print_state_features(Counter(g_crf.state_features_).most_common()[:5])


Top positive:
5.075940 O        word[-3:]:day
4.939286 B-PER    lower:clinton
4.839990 B-ORG    -1:word.lower():v
4.778240 B-LOC    +1:word.lower():1996-08-26
4.612259 B-LOC    +1:word.lower():1996-08-27


### Random Search CV

In [ ]:
import scipy
params_space = {
 'c1': scipy.stats.expon(scale=0.5),
 'c2': scipy.stats.expon(scale=0.05),
}

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer
f1_scorer = make_scorer(metrics.flat_f1_score, average='macro')
rs = RandomizedSearchCV(crf, params_space,
 cv=3,
 verbose=1,
 n_jobs=-1,
 n_iter=50,
 scoring=f1_scorer)


In [ ]:
rs.fit(X_train, y_train)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:213: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed: 21.8min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed: 69.6min finished


RandomizedSearchCV(cv=3,
                   estimator=CRF(algorithm='lbfgs',
                                 all_possible_transitions=False, c1=0.1, c2=0.1,
                                 keep_tempfiles=None, max_iterations=100),
                   n_iter=50, n_jobs=-1,
                   param_distributions={'c1': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f93c70a3890>,
                                        'c2': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f93c70a3a90>},
                   scoring=make_scorer(flat_f1_score, average=macro),
                   verbose=1)

In [ ]:
crf = rs.best_estimator_
print('best params:', rs.best_params_)
print('best CV score:', rs.best_score_)

best params: {'c1': 0.010537878886435896, 'c2': 0.03659085667683192}
best CV score: 0.8453897261505215


In [ ]:
print("\nTop unlikely transitions:")
print_transitions(Counter(crf.transition_features_).most_common()[-5:])


Top unlikely transitions:
I-MISC -> B-PER   -2.560967
I-ORG  -> B-PER   -2.692467
B-ORG  -> B-LOC   -3.210803
I-ORG  -> B-ORG   -3.837738
I-ORG  -> B-LOC   -4.165044


In [ ]:
print("\nTop negative:")
print_state_features(Counter(crf.state_features_).most_common()[-5:])


Top negative:
-4.167455 O        -1:word.lower():200
-4.239530 O        -1:word.lower():lloyd
-4.245181 B-ORG    -1:word.lower():russian
-5.192148 O        +1:word.lower():arose
-5.918498 O        is_title
